In [3]:
import torch
import albumentations as A
import numpy as np
from PIL import Image

def augment_one_image(image, device, num_augmentations=5):
    """对图像进行数据增强，每次应用不同的增强方式，生成多个增强图像"""
    
    # 定义每种单一的增强操作
    augmentations = [
        A.Compose([A.HorizontalFlip(p=1)]),  # 水平翻转
        A.Compose([A.VerticalFlip(p=1)]),  # 垂直翻转
        A.Compose([A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.3, rotate_limit=30, p=1)]),  # 平移和旋转
        A.Compose([A.RandomBrightnessContrast(p=1)]),  # 亮度和对比度调整
        A.Compose([A.Blur(blur_limit=3, p=1)]),  # 模糊
    ]
    
    augmented_images = []

    # 将输入图像转换为Tensor，并移到GPU上进行处理
    image_tensor = torch.from_numpy(image).to(device)

    for i in range(num_augmentations):
        # 使用不同的增强操作
        augmentation_pipeline = augmentations[i]
        augmented = augmentation_pipeline(image=image_tensor.cpu().numpy())
        augmented_image = augmented['image']
        augmented_images.append(augmented_image)
    
    return augmented_images
image = np.array(Image.open("11.png"))  # 加载图像
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # 选择设备
augmented_images = augment_one_image(image, device)

# 保存增强后的图像
for idx, aug_image in enumerate(augmented_images):
    aug_image_pil = Image.fromarray(aug_image)
    aug_image_pil.save(f"augmented_image_{idx}.png")
